In [8]:
import os
from PIL import Image
import numpy as np
import pickle
import cv2
from mtcnn.mtcnn import MTCNN
# from calibration import Calibration
import dlib
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.animation import FuncAnimation
import csv
%matplotlib notebook

In [9]:
from keras.models import load_model
model = load_model('./trained_models/model.h5')
model_eye = load_model('./trained_models/eye_model.h5')
# calibration = Calibration()
_predictor = dlib.shape_predictor("trained_models/shape_predictor_68_face_landmarks.dat")
detector = MTCNN()

In [10]:
def get_pixele_test(img_array, required_size=(60, 60)):
    image = Image.fromarray(img_array)
    image = image.resize(required_size)
    face_array = np.asarray(image)/255
    return face_array

def preprocess_img(img):
    img = Image.fromarray(img)
    img = img.resize((30,30))
    img = np.array(img)
    img = img/255
    img = img.reshape(30, 30, 1)
    img = np.expand_dims(img,axis=0)
    return img

In [11]:
def prediction_eyes(faces,frame_g):
    bounding_box = faces[i]['box']
    x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
    face = dlib.rectangle(x1,y1,x1+width,y1+height)
    landmarks = _predictor(frame, face)
    left_eye = []
    right_eye = []
    left_eye.append([landmarks.part(36).x,landmarks.part(36).y])
    left_eye.append([landmarks.part(37).x,landmarks.part(37).y])
    left_eye.append([landmarks.part(38).x,landmarks.part(38).y])
    left_eye.append([landmarks.part(39).x,landmarks.part(39).y])
    left_eye.append([landmarks.part(40).x,landmarks.part(40).y])
    left_eye.append([landmarks.part(41).x,landmarks.part(41).y])

    right_eye.append([landmarks.part(42).x,landmarks.part(42).y])
    right_eye.append([landmarks.part(43).x,landmarks.part(43).y])
    right_eye.append([landmarks.part(44).x,landmarks.part(44).y])
    right_eye.append([landmarks.part(45).x,landmarks.part(45).y])
    right_eye.append([landmarks.part(46).x,landmarks.part(46).y])
    right_eye.append([landmarks.part(47).x,landmarks.part(47).y])

    left_eye = np.array(left_eye)
    right_eye = np.array(right_eye)

    rect_left = cv2.boundingRect(left_eye)
    x,y,w,h = rect_left
    txt_left = (x,y)
    croped_left = frame_g[y:y+h, x:x+w].copy()

    left_eye = left_eye - left_eye.min(axis=0)
    cv2.polylines(croped_left, np.array([left_eye], int), True, (0,0,0), 2)
    mask_left = np.zeros(croped_left.shape[:2], np.uint8)
    cv2.drawContours(mask_left, [left_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
    dst_left = cv2.bitwise_and(croped_left, croped_left, mask=mask_left)
#         cv2.imshow("left",dst_left)

    rect_right = cv2.boundingRect(right_eye)
    x,y,w,h = rect_right
    txt_right = (x,y)
    croped_right = frame_g[y:y+h, x:x+w].copy()
    right_eye = right_eye - right_eye.min(axis=0)
    cv2.polylines(croped_right, np.array([right_eye], int), True, (0,0,0), 2)
    mask_right = np.zeros(croped_right.shape[:2], np.uint8)
    cv2.drawContours(mask_right, [right_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
    dst_right = cv2.bitwise_and(croped_right, croped_right, mask=mask_right)
#         cv2.imshow("right",dst_right)
    left_eye_img = preprocess_img(dst_left)
    right_eye_img = preprocess_img(dst_right)

    left_pred = model_eye.predict(left_eye_img)
    right_pred = model_eye.predict(right_eye_img)
    pred_eye = []
    pred_eye.append(left_pred)
    pred_eye.append(right_pred)
    return pred_eye


In [12]:
# cam = cv2.VideoCapture(0)
# font = cv2.FONT_HERSHEY_SIMPLEX 
 
# fontScale = 1
# color = (255, 0, 0) 
# thickness = 2

# while(True):
#     ret,frame = cam.read()
#     key_pressed = cv2.waitKey(1)&0xFF
#     if ret==False:
#         print("Something went wrong")
#         continue
#     if key_pressed == ord('q'):
#         break
#     frame = Image.fromarray(frame)
#     frame = frame.resize((640,480))
#     frame = np.array(frame)
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# #     frame = cv2.flip(frame, 1)
#     faces = detector.detect_faces(frame)
#     for i in range(len(faces)):
#         bb = faces[i]['box']
#         x, y, w, h = bb
#         f_img = frame[y:y+h,x:x+w]
#         face = get_pixele_test(f_img)
#         face = np.expand_dims(face, axis=0)
#         pred = model.predict(face)
        
#         if pred>0.5:
#             frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (255, 0, 0), thickness)
            
#         else:
#             frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (0, 255, 0), thickness)
            
#             pred_eyes = prediction_eyes(faces,frame_g)
# #             print(pred_eyes)
#             if pred_eyes[0]>=0.9 and pred_eyes[1]>=0.9:
#                 cv2.putText(frame,"cheating_eyes",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
#             else:
#                 cv2.putText(frame,"Ok",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
    
    
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     cv2.imshow('yo', frame)
# cv2.waitKey()
# cv2.destroyAllWindows()
    

In [13]:
n=4
logs = {}
for i in range(n):
    logs[i] = []

In [14]:
def calculate_logs(n,pred,time):
    global logs
    if(pred>0.5):
        logs[n].append([1,time])
    else:
        logs[n].append([0,time])
    

In [15]:
bar_val = []
for i in range(n):
    bar_val.append(0)

plt.style.use('fivethirtyeight')
def plot_log():
    global logs
    global bar_val
    global ypos
    bar_val = [0,0,0,0]
    for k in logs.keys():
        l = logs[k]
        time = 0
        c_time = 0
        for j in range(len(l)):
            if(l[j][0]==1):
                c_time = l[j][1]
            elif c_time!=0:
                time = time + (l[j][1] - c_time).total_seconds()
                c_time = 0
        
        bar_val[k] = bar_val[k] + time
    out = csv.writer(open("./final_dataset/bar_graph.csv","w"), delimiter=',',quoting=csv.QUOTE_ALL)
    out.writerow(bar_val)
#     print(bar_val)
    


In [16]:

cams = []
path = "./final_dataset/test"
for root,dirs,files in os.walk(path):
    for file in files:
        cams.append(cv2.VideoCapture(path+'/'+file))
font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 1
color = (255, 0, 0) 
thickness = 2
col = 0
row_init = 0


while(True):
    rows = []
    for n,cam in enumerate(cams):  
        
        ret,frame = cam.read()
        
        key_pressed = cv2.waitKey(1)&0xFF
    #     frame = final_frame
        if ret==False:
            print("Something went wrong")
            continue
        if key_pressed == ord('q'):
            break
        frame = Image.fromarray(frame)
        frame = frame.resize((640,480))
        frame = np.array(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #     frame = cv2.flip(frame, 1)
        faces = detector.detect_faces(frame)
    #     print(len(faces))
        for i in range(len(faces)):
            bb = faces[i]['box']
            x, y, w, h = bb
            f_img = frame[y:y+h,x:x+w]
            face = get_pixele_test(f_img)
            face = np.expand_dims(face, axis=0)
            pred = model.predict(face)
            
            calculate_logs(n,pred,datetime.now())
            plot_log()
            
            if pred>0.5:
                frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (255, 0, 0), thickness)
            else:
                frame = cv2.rectangle(frame, (int(x),int(y)), (int(x+w),int(y+h)), (0, 255, 0), thickness)
                pred_eyes = prediction_eyes(faces,frame_g)
    #             print(pred_eyes)
                if pred_eyes[0]>=0.5 and pred_eyes[1]>=0.5:
                    cv2.putText(frame,"cheating_eyes",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)
                else:
                    cv2.putText(frame,"Ok",(int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 1, cv2.LINE_AA)

        frame = Image.fromarray(frame)
        frame = frame.resize((300,300))
        frame = np.array(frame)
                    
        if(col<2):
            if(col==0):
                row = frame
                col +=1
            else:
                row = np.hstack((row,frame))
                col +=1
        if(col==2):
            rows.append(row)
            col=0
    final_frame = rows[0]
    for i in range(1,len(rows)):
        final_frame = np.vstack((final_frame,rows[i]))
    final_frame = cv2.cvtColor(final_frame, cv2.COLOR_BGR2RGB)
    cv2.imshow("final frame", final_frame)
cv2.waitKey()
cv2.destroyAllWindows()
    

KeyboardInterrupt: 